In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from sklearn.utils import shuffle

import keras


sns.set(style='white', context='notebook', palette='deep')

In [ ]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

#### Here, we are just setting the Y and X from the training dataset. 

In [ ]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

sns.countplot(Y_train)

Y_train.value_counts()

#### Performing GrayScale Normalization

In [ ]:
X_train = X_train / 255.0
test = test / 255.0

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [ ]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [ ]:
g = plt.imshow(X_train[0][:,:,0])

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=42)

### Building my own model + Data Augmentation

#### Data Augmentation is the process of changing or adding noise to image

In [ ]:
# A helper function to create augmentation of images
def augmentImages(images, multiple):
    gen = ImageDataGenerator()
    newImages = []
    for _ in range(multiple):
        for image in images:
            transformedImage = gen.apply_transform(image, {'brightness': np.random.uniform(low = 0.5, high=0.99),
                                                            'tx': np.random.random_integers(2),
                                                            'ty': np.random.random_integers(2)})
            newImages.append(transformedImage)
    newImages = np.array(newImages)
    return newImages

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (2,2), activation='relu'))
model.add(Conv2D(128,(2,2), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(1e-3),
              loss = 'categorical_crossentropy',
              metrics=['accuracy']
             )

In [ ]:
newNumbers = augmentImages([X_train[0]],1)

In [ ]:
plt.imshow(newNumbers.reshape(28,28))

In [ ]:
plt.imshow(X_train[0].reshape(28,28))

In [ ]:
augmentPic = augmentImages(X_train,1)

In [ ]:
x_train, y_train = shuffle(X_train, Y_train)


In [ ]:
model.fit(x_train, y_train, batch_size = 32, epochs=50)

In [ ]:
score = model.evaluate(X_val, Y_val, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy', score[1])